In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import colorlover as cl

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Bar, Scatter, Figure, Layout, Margin, Data, Scattermapbox, Marker
init_notebook_mode(connected=True)

from difflib import SequenceMatcher

pd.options.display.max_rows = 9999
pd.options.display.max_columns = 99

colors = cl.scales['5']['qual']['Paired']

# BiciMAD
## Usage and problems
<br><br>

## Introduction

The aim of the following document is to explore the usage BiciMAD.

BiciMAD is a public bike rental service that is available in Madrid. The use is very simple: there are several bases distributed accross Madrid, and it is possible to rent a bike from one of the bases and return it in another. 

To be able to rent a bike, the users need to be either anual subscriptors (anual owners) or occasional users. The payment can be done in the base themselves prior to the bike extraction. The anual owners acquire a card that can be charged and use to pay the bike extraction. 

The pricing of the service has several rules, but basically the user has to pay 50 cents for the first 30 minutes of usage and 60 cents for the following ones. There is an extra charge if the user drops the bike in a crawdled station, and there's also a discount if the user drops the bike in an empty one. In order to make sure the user can drop the bike in a base (spots will be available) it is possible to reserve them with an additional cost.


In [2]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="images/sol.jpg", width=1000)

## Daily Active Users

Let's start with a description of the BiciMAD users since the beggining of the service in June 2014.

In the following graph both anual and occasional users are presented:

In [3]:
df_bikes_bm = pd.read_excel("data/bici_usos_acumulados201703.xls")
df_bikes_bm = df_bikes_bm[df_bikes_bm.columns[:4]]#, #"Usos bicis aanual anual"]]#, "Usos bicis aanual ocasional"]]
df_bikes_bm.columns = ["date", "anual", "occasional", "total"]
df_bikes_bm.fillna(0, inplace=True)
df_bikes_bm["day"] = df_bikes_bm["date"].apply(lambda date: date.weekday())
df_bikes_bm["month"] = df_bikes_bm["date"].apply(lambda date: date.month)

In [4]:
data = [
    Scatter(
        x=df_bikes_bm.date, 
        y=df_bikes_bm.anual,
        name="anual user",
    ),
    Scatter(
        x=df_bikes_bm.date, 
        y=df_bikes_bm.occasional,
        name="occasional user",
    ),
]
layout = Layout(
    yaxis=dict(title="DAU"),
    title='BiciMAD Daily Active Users',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

The first thing that can be noticed is the seasonality of the service, describing peaks during spring time and valleys in winter. There's also a little local valley during summer time.

The second thing worth mentioning is that although the growth was fast at first, the user acqusition got stuck after the second year, and the churn is increasing after 2016 (DAU is decreasing).

And finally, we can see that the occasional users are much lower that the anual users, which indicates that BiciMAD is failing at atracting them.

Let's now explore when the users use BiciMAD, weekly and monthly:

In [5]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
data = [
    Bar(
        x=[days[day] for day in df_bikes_bm.groupby("day").agg({"anual": "mean"}).index], 
        y=df_bikes_bm.groupby("day").agg({"anual": "mean"}).anual,
        name="anual user",
    ),
    Bar(
        x=[days[day] for day in df_bikes_bm.groupby("day").agg({"occasional": "mean"}).index], 
        y=df_bikes_bm.groupby("day").agg({"occasional": "mean"}).occasional,
        name="occasional user",
    ),
]
layout = Layout(
    barmode='stack',
    yaxis=dict(title="users"),
    title='BiciMAD users per day of the week',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

months = ["", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
data = [
    Bar(
        x=[months[month] for month in df_bikes_bm.groupby("month").agg({"anual": "mean"}).index], 
        y=df_bikes_bm.groupby("month").agg({"anual": "mean"}).anual,
        name="anual user",
    ),
    Bar(
        x=[months[month] for month in df_bikes_bm.groupby("month").agg({"occasional": "mean"}).index], 
        y=df_bikes_bm.groupby("month").agg({"occasional": "mean"}).occasional,
        name="occasional user",
    ),
]
layout = Layout(
    barmode='stack',
    yaxis=dict(title="users"),
    title='BiciMAD users per month',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

Occasional users use BiciMAD in the weekends, and anual users use it during the week, which indicate that many anual users use BiciMAD to commute, but still many of them use the service during the weekend.

For the monthly usage, we can now see the seasonality with more detail. There is a peak in spring and a local valley in summer, that can be due to excesive hot weather, or just people leaving Madrid for holidays. Interestingly, for occasional users the peak is also in May, and there's also a valley in July. Another interesting peak appears in December for occasional users.

## Bases and spots

Let's now explore the base distribution per district. There are 4146 spots distributed in 171 bases accross 8 district:

In [6]:
df_bikes_bases = pd.read_csv("data/bases_bicimad.csv", sep=";", encoding="ISO-8859-1")
df_bikes_bases["Fecha de Alta"] = df_bikes_bases["Fecha de Alta"]\
                                        .apply(lambda date: datetime.strptime(date, "%d/%m/%Y"))

In [7]:
df_bikes_bases_district = df_bikes_bases.groupby("Distrito", as_index=False)\
                                        .agg({"Número": "count", "Número de Plazas": "sum"})
    
df_bikes_bases_district.columns = ["district", "bases", "spots"]
df_bikes_bases_district.district = df_bikes_bases_district.district.apply(lambda x: x[2:])
df_bikes_bases_district.sort_values("bases", ascending=False, inplace=True)

In [8]:
data = [
    Bar(
        x=df_bikes_bases_district.district, 
        y=df_bikes_bases_district.bases,
        name="Bases"
    ),
    Scatter(
        x=df_bikes_bases_district.district, 
        y=df_bikes_bases_district.spots,
        name="Spots",
        yaxis="y2"
    )
]
layout = Layout(
    #barmode="stack",
    yaxis=dict(title="bases"),
    yaxis2=dict(title="spots", side="right", overlaying='y'),
    title='BiciMAD Bases and Spots',
    margin=Margin(b=150)
)
fig = Figure(data=data, layout=layout)
iplot(fig)

As we can see the bases are unevenly distributed, and they're more condensed around the city centre, Salamanca and Retiro districts. The main problem we could face with an uneven distrivution is that commuters living outside the center may have problems finding a station close to their homes. 

Let's observe the growing of the bases:

In [9]:
df_bikes_bases_date = df_bikes_bases.groupby("Fecha de Alta", as_index=False)\
                                    .agg({"Número": "count", "Número de Plazas": "sum"}).sort_index()
    
df_bikes_bases_date.columns = ["date", "bases", "spots"]
df_bikes_bases_date["bases_cumsum"] = df_bikes_bases_date.bases.cumsum()
df_bikes_bases_date["spots_cumsum"] = df_bikes_bases_date.spots.cumsum()

In [10]:
data = [
    Bar(
        x=df_bikes_bases_date.date, 
        y=df_bikes_bases_date.bases,
        name="New bases"
    ),
    Scatter(
        x=df_bikes_bases_date.date, 
        y=df_bikes_bases_date.bases_cumsum,
        name="Total bases",
        #yaxis="y2"
    )
]
layout = Layout(
    #barmode="stack",
    yaxis=dict(title="bases"),
    #yaxis2=dict(title="spots", side="right", overlaying='y'),
    title='BiciMAD New Bases',
    margin=Margin(b=150)
)
fig = Figure(data=data, layout=layout)
iplot(fig)

data = [
    Bar(
        x=df_bikes_bases_date.date, 
        y=df_bikes_bases_date.spots,
        name="New spots"
    ),
    Scatter(
        x=df_bikes_bases_date.date, 
        y=df_bikes_bases_date.spots_cumsum,
        name="Total spots",
        #yaxis="y2"
    )
]
layout = Layout(
    #barmode="stack",
    yaxis=dict(title="spots"),
    #yaxis2=dict(title="spots", side="right", overlaying='y'),
    title='BiciMAD New spots',
    margin=Margin(b=150)
)
fig = Figure(data=data, layout=layout)
iplot(fig)

As we can see there are two only points of growing, at the beginning of the service, and one year later. There is not a significant amount of new bases after that, which could be a problem if we remember that the DAU was decreasing precisely after 2015.

Let's see the availability of bikes per number of hours of use and disponibility:

In [11]:
df_bike_usage = pd.read_csv("data/bici_disponibilidad20170607.csv", sep=";")
df_bike_usage.columns = ["date", "hours_usage", "hours_available", "hours_total", 
                         "bikes_available", "use_anual", "use_occasional", "use_total"]

df_bike_usage["date"] = df_bike_usage["date"].apply(lambda date: datetime.strptime(date, "%d/%m/%Y"))

data = [
    Scatter(
        x=df_bike_usage.date, 
        y=df_bike_usage.hours_available,
        name="hours available",
    ),
    Scatter(
        x=df_bike_usage.date, 
        y=df_bike_usage.hours_usage,
        name="hours usage",
    ),
    Scatter(
        x=df_bike_usage.date, 
        y=df_bike_usage.bikes_available,
        name="bikes available",
        yaxis="y2"
    ),
]
layout = Layout(
    yaxis=dict(title="hours"),
    yaxis2=dict(title="bikes", side="right", overlaying='y'),
    title='BiciMAD daily users',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

We can see that there's a huge dissonance between the available number of bikes and the bikes usage. This indicates that a big number of bikes remain unused along the day. There's a big drop in the availability in November 2016, together with a steady drop in the bikes usage. In November 6th of the same year, the number of bikes available crossed the bikes usage (hours).

It is difficult to know if there's a demand of bikes or, on the contrary, there are more than enough bikes. In any case, having too many bikes may be a problem because some of them may not be on optimal conditions, in stations far away where they are not needed, or even in centric stations occupying spots that will make impossible for users to park their bikes. In any case it is a waste of resources.

## BiciMAD reporting tickets

To end exploring BiciMAD data, let's take a look at the issues that users face when using the service. This are tickets that can be issue through City Hall phone number, the website, or BiciMAD APP. 

There are three kinds of tickets: Bike related, Signs related, and SER (parking related). 

In [12]:
df_bike_issues = pd.read_csv("data/bici_incidencias20170607.csv", sep=";", encoding="ISO-8859-1")

In [24]:
df_bike_issues_g = df_bike_issues.groupby(["SECCION_ANOMALIA"], as_index=False)\
                                 .agg({"RECEPCIONADAS": "sum", "ABIERTAS": "sum", "CERRADAS": "sum"})

data = [
    Bar(
        x=df_bike_issues_g.SECCION_ANOMALIA.iloc[:-1], 
        y=df_bike_issues_g.RECEPCIONADAS.iloc[:-1],
        name="Received"
    ),
    Bar(
        x=df_bike_issues_g.SECCION_ANOMALIA.iloc[:-1], 
        y=df_bike_issues_g.ABIERTAS.iloc[:-1],
        name="Open"
    ),
    Bar(
        x=df_bike_issues_g.SECCION_ANOMALIA.iloc[:-1], 
        y=df_bike_issues_g.CERRADAS.iloc[:-1],
        name="Closed"
    )
]
layout = Layout(
    barmode="stack",
    title='Customer Care tickets',
    margin=Margin(b=150),
    yaxis=dict(title="tickets")
)
fig = Figure(data=data, layout=layout)
iplot(fig)

data = [
    Bar(
        x=df_bike_issues_g.SECCION_ANOMALIA.iloc[-1:], 
        y=df_bike_issues_g.ABIERTAS.iloc[-1:],
        name="Open"
    ),
    Bar(
        x=df_bike_issues_g.SECCION_ANOMALIA.iloc[-1:], 
        y=df_bike_issues_g.CERRADAS.iloc[-1:],
        name="Closed",
    )
]
layout = Layout(
    title='Total Customer Care tickets',
    margin=Margin(b=150),
    yaxis=dict(title="tickets"),
    annotations=[
        dict(
            x="TOTAL",
            y=150000,
            xshift=80,
            xanchor='left',
            yanchor='bottom',
            showarrow=False,
            text="Unclosed tickets: {}"\
                    .format(df_bike_issues_g.iloc[-1]["ABIERTAS"] - df_bike_issues_g.iloc[-1]["CERRADAS"]),
        )
    ]
)
fig = Figure(data=data, layout=layout)
iplot(fig)

As we can see most of the tickets are bike related, and the vast majority remains open, which means basically that there was no solution presented to the user. That is a huge number of unhappy bikers!

Let's see the trend over the last two years:

In [25]:
data = [
    Scatter(
        x=df_bike_issues[df_bike_issues.SECCION_ANOMALIA==section].DIA_INICIO, 
        y=df_bike_issues[df_bike_issues.SECCION_ANOMALIA==section].ABIERTAS,
        name="Open - {}".format(section),
        line=dict(color=colors[i])
    ) 
    for i, section in enumerate(df_bike_issues.SECCION_ANOMALIA.unique())
] + [
    Scatter(
        x=df_bike_issues[df_bike_issues.SECCION_ANOMALIA==section].DIA_INICIO, 
        y=df_bike_issues[df_bike_issues.SECCION_ANOMALIA==section].CERRADAS,
        name="Closed - {}".format(section),
        line=dict(color=colors[i], dash="dash")
    ) 
    for i, section in enumerate(df_bike_issues.SECCION_ANOMALIA.unique())
]
layout = Layout(
    yaxis=dict(title="tickets"),
    title='BiciMAD daily tickets',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

The most interesting pattern to notice in the trend is the delay closing the tickets. A peak in open tickets is followed by a peak in closed tickets one or two weeks later. At least that is the trend at the beginning of the period. After 2016 the number of unresolved tickets start increasing, and peaks in received tickets is not followed by peaks in closed ones. The tendency comes back in May 2017.

# Traffic accidents involving bikes

Let's now change the topic from BiciMAD to general bike usage, particularly, let's examine the traffic accidents involving bikes in Madrid. The available data starts in 2017. 

The accidents are distributed in categories: collisions, bike falling, run over (pedestrians), crash agains fixed objects and others:

In [15]:
def get_date(row):
    return datetime.strptime(row["Fecha"], "%d/%m/%Y %H:%M").date()

def get_time(row):    
    return datetime.strptime(row["TRAMO HORARIO"].split(" ")[1], "%H:%M").time()

def get_day(row):    
    day = row["date"].weekday()
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return day
    #return days[day]
    
    
accident_dictionary = {
    "CHOQUE CON OBJETO FIJO": "Crash against fixed object",
    "CAÍDA BICICLETA": "Bike falling",
    "ATROPELLO": "Run over",
    "COLISIÓN DOBLE": "Double collision",
    "CAÍDA MOTOCICLETA": "Other",
    "CAÍDA VIAJERO BUS": "Other"
}

df_bike_crashes = pd.read_csv("data/AccidentesBicicletas_2017.csv", sep=";", encoding="ISO-8859-1")
df_bike_crashes = df_bike_crashes.iloc[:202]
df_bike_crashes["date"] = df_bike_crashes.apply(get_date, axis=1)
df_bike_crashes["time"] = df_bike_crashes.apply(get_time, axis=1)
df_bike_crashes["day"] = df_bike_crashes.apply(get_day, axis=1)


df_bike_crashes = df_bike_crashes[["Nm Tot Victimas", "DISTRITO", "Tipo Accidente", "date", "time", "day"]]
df_bike_crashes.columns = ["victims", "district", "type", "date", "time", "day"]
df_bike_crashes["type"] = df_bike_crashes["type"].str.strip().map(accident_dictionary)

In [16]:
total_days = (df_bike_crashes.date.iloc[-1] - df_bike_crashes.date.iloc[0]).days
total_victims = df_bike_crashes["victims"].sum()

victims_per_day = total_victims / total_days

In [17]:
data = [
    Bar(
        x=df_bike_crashes.groupby("type").agg({"victims": "sum"}).sort_values("victims", ascending=False).index, 
        y=df_bike_crashes.groupby("type").agg({"victims": "sum"}).sort_values("victims", ascending=False).victims,
        name="Victims"
    ),
    Scatter(
        x=df_bike_crashes.groupby("type").agg({"victims": "count"}).sort_values("victims", ascending=False).index, 
        y=df_bike_crashes.groupby("type").agg({"victims": "count"}).sort_values("victims", ascending=False).victims,
        name="Accidents",
        yaxis="y2"
    )
]
layout = Layout(
    yaxis=dict(title="Victims"),
    yaxis2=dict(title="Accidents", side="right", overlaying='y'),
    title='Total victims distributed by type',
    margin=Margin(b=150)
)
fig = Figure(data=data, layout=layout)
iplot(fig)

Most of the accidents are produced by collisions and bike falling, followed by crashes involving pedestrians. 

Double collision crashes could be caused by both cars and bikes not following traffic rules. Bike falling crashes could be caused by a driver mistake, but also by road/bike conditions. This data is not exclusive from BiciMAD, but it reinforce the need of maintenance in the public service. About the running over crashes, it is probably caused by both, driving along the side walk, and crossing pedestrian paths at red light. 

In [26]:
data = [
    Bar(
        x=df_bike_crashes[df_bike_crashes.type==acc_type].groupby("date").agg({"victims": "sum"}).index, 
        y=df_bike_crashes[df_bike_crashes.type==acc_type].groupby("date").agg({"victims": "sum"}).victims,
        name=acc_type
    ) 
    for acc_type in df_bike_crashes["type"].unique()
]
layout = Layout(
    yaxis=dict(title="victims"),
    title='Total victims evolution',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

Let's examine now the accidents according to the time of the day and the day of the week they occur.

In [27]:
data = [
    Bar(
        x=df_bike_crashes.groupby("time").agg({"victims": "sum"}).index, 
        y=df_bike_crashes.groupby("time").agg({"victims": "sum"}).victims,
    )
]
layout = Layout(
    yaxis=dict(title="victims"),
    title='Total victims distribution by hour',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

We can see that most of the accidents occur from 14:00 to 20:00, being 14:00 the highest peak. The peaks at 14:00, 16:00 and 19:00-20:00 may indicate rush hours and people commuting. Let's see if this is reflected on the distribution by day:

In [28]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
data = [
    Bar(
        x=[days[day] for day in df_bike_crashes.groupby("day").agg({"victims": "sum"}).index], 
        y=df_bike_crashes.groupby("day").agg({"victims": "sum"}).victims,
    )
]
layout = Layout(
    yaxis=dict(title="victims"),
    title='Total victims distribution by day',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

The accidents are actually evendly distributed, but there are three peaks on Tuesday, Thursday and Friday. However the number of accidents does not decrease that much on the weekends. Finally, let's see how the accidents are distributed by district.

In [29]:
data = [
    Bar(
        x=df_bike_crashes.groupby("district").agg({"victims": "sum"})\
                         .sort_values("victims", ascending=False).index, 
        y=df_bike_crashes.groupby("district").agg({"victims": "sum"})\
                         .sort_values("victims", ascending=False).victims,
    )
]
layout = Layout(
    yaxis=dict(title="victims"),
    title='Total victims distribution by district',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

# Insights

As we have seen BiciMAD is well established since they launched the service in June 2014. However, they should take care of the **dropping rate of anual users**, and definetely look into the **unattended occasional users**, who may not been able to find in BiciMAD the service they need. They should start looking into both, the **difference between the number of available bikes and bike usage**, and the **Customer Care** service, which is clearly underperforming in terms of closing tickets. 

About the safety when riding bikes, as we have seen **the majority of accidents involve collisions** with other vehicles. Further research is definetely needed, and looking into how bikes and cars interact is probably a good start. 
<br><br><br><br>

In [22]:
Image(url="images/alcala.jpg", width=1000)

<br><br><br><br><br><br><br><br>

In [23]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
</form>''')

2017, Pablo Bordons Estrada